In [1]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

C:\ProgramData\Anaconda2\envs\dlnd\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# define function to load train, test, and validation datasets
def load_dataset(path, number_of_classes):
    data = load_files(path)
    x = np.array(data['filenames'])
    y = np_utils.to_categorical(np.array(data['target']), number_of_classes)
    return x, y

In [3]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Input, Conv2D, MaxPooling2D, GlobalMaxPooling2D

In [4]:
train_data_dir = "./data/train"
validation_data_dir = "./data/valid"
test_data_dir = "./data/test"
number_of_classes = 3
# load list of dog names
tissue_names = [item[20:-1] for item in sorted(glob("./data/train/*/"))]
img_width, img_height = 3008, 2000
target_width, target_height = 300, 300

In [5]:
# load train, test, and validation datasets
train_files, train_targets = load_dataset(train_data_dir, number_of_classes)
valid_files, valid_targets = load_dataset(validation_data_dir, number_of_classes)
test_files, test_targets = load_dataset(test_data_dir, number_of_classes)

In [6]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(target_height, target_width))
    # convert PIL.Image.Image type to 3D tensor with shape (400, 400, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, target_height, target_width, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [7]:
print('There are %d total tissue categories.' % len(tissue_names))
print('There are %s total tissue images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training tissue images.' % len(train_files))
print('There are %d validation tissue images.' % len(valid_files))
print('There are %d test tissue images.'% len(test_files))

There are 3 total tissue categories.
There are 2750 total tissue images.

There are 2000 training tissue images.
There are 150 validation tissue images.
There are 600 test tissue images.


In [8]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [05:53<00:00,  1.70it/s]


In [59]:
### Build the network
from keras.layers.normalization import BatchNormalization

model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(target_height, target_width, 3), name='conv_1'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='pool_1'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='conv_2'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='pool_2'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='conv_3'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='pool_3'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='conv_4'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='pool_4'))
#model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='conv_5'))
#model.add(BatchNormalization())
#model.add(Dropout(0.35))
#model.add(MaxPooling2D((2, 2), strides=(2, 2), name='pool_5'))
#model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='conv_6'))
#model.add(Dropout(0.35))
#model.add(MaxPooling2D((2, 2), strides=(2, 2), name='pool_6'))

# Classification
model.add(GlobalMaxPooling2D())
model.add(Dense(150, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
#model.add(Dense(100, activation='relu'))
#model.add(Dropout(0.25))
model.add(Dense(3, activation='softmax'))


model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 300, 300, 64)      1792      
_________________________________________________________________
batch_normalization_57 (Batc (None, 300, 300, 64)      256       
_________________________________________________________________
dropout_63 (Dropout)         (None, 300, 300, 64)      0         
_________________________________________________________________
pool_1 (MaxPooling2D)        (None, 150, 150, 64)      0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 150, 150, 64)      36928     
_________________________________________________________________
batch_normalization_58 (Batc (None, 150, 150, 64)      256       
_________________________________________________________________
dropout_64 (Dropout)         (None, 150, 150, 64)      0         
__________

In [60]:
# Compile model
model.compile(loss = "categorical_crossentropy", optimizer='adam', metrics=["accuracy"])

In [61]:
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='saved_models/weights.cancer.hdf5', 
                               verbose=1, save_best_only=True)

from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        #rotation_range=5,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

epochs = 15

In [62]:
# Fit model on training data
#model.fit(train_tensors, train_targets, 
#          validation_data=(valid_tensors, valid_targets),
#          epochs=epochs, batch_size=2, callbacks=[checkpointer], verbose=1)

model.fit_generator(datagen.flow(train_tensors, train_targets, batch_size=12), 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, callbacks=[checkpointer], verbose=1)

Epoch 1/15
167/167 [==============================] - ETA: 7:53 - loss: 1.5853 - acc: 0.250 - ETA: 4:45 - loss: 1.4887 - acc: 0.375 - ETA: 3:46 - loss: 1.4933 - acc: 0.305 - ETA: 3:12 - loss: 1.5512 - acc: 0.312 - ETA: 2:51 - loss: 1.4731 - acc: 0.366 - ETA: 2:37 - loss: 1.4523 - acc: 0.388 - ETA: 2:27 - loss: 1.4550 - acc: 0.357 - ETA: 2:20 - loss: 1.4383 - acc: 0.364 - ETA: 2:13 - loss: 1.5028 - acc: 0.370 - ETA: 2:08 - loss: 1.4967 - acc: 0.358 - ETA: 2:04 - loss: 1.4985 - acc: 0.348 - ETA: 2:01 - loss: 1.4949 - acc: 0.347 - ETA: 1:57 - loss: 1.4731 - acc: 0.359 - ETA: 1:55 - loss: 1.4645 - acc: 0.357 - ETA: 1:52 - loss: 1.4438 - acc: 0.366 - ETA: 1:50 - loss: 1.4040 - acc: 0.380 - ETA: 1:48 - loss: 1.4105 - acc: 0.372 - ETA: 1:46 - loss: 1.3866 - acc: 0.384 - ETA: 1:44 - loss: 1.3681 - acc: 0.394 - ETA: 1:42 - loss: 1.3759 - acc: 0.391 - ETA: 1:41 - loss: 1.3652 - acc: 0.388 - ETA: 1:39 - loss: 1.3649 - acc: 0.386 - ETA: 1:38 - loss: 1.3513 - acc: 0.391 - ETA: 1:37 - loss: 1.3382 -

167/167 [==============================] - ETA: 1:34 - loss: 1.1033 - acc: 0.583 - ETA: 1:34 - loss: 1.0336 - acc: 0.583 - ETA: 1:34 - loss: 0.9922 - acc: 0.611 - ETA: 1:33 - loss: 1.0445 - acc: 0.562 - ETA: 1:33 - loss: 1.0199 - acc: 0.566 - ETA: 1:32 - loss: 0.9812 - acc: 0.583 - ETA: 1:31 - loss: 0.9621 - acc: 0.571 - ETA: 1:31 - loss: 0.9690 - acc: 0.583 - ETA: 1:30 - loss: 0.9172 - acc: 0.611 - ETA: 1:30 - loss: 0.8999 - acc: 0.625 - ETA: 1:29 - loss: 0.9235 - acc: 0.613 - ETA: 1:29 - loss: 0.9233 - acc: 0.618 - ETA: 1:28 - loss: 0.9211 - acc: 0.615 - ETA: 1:27 - loss: 0.9177 - acc: 0.619 - ETA: 1:27 - loss: 0.9327 - acc: 0.616 - ETA: 1:26 - loss: 0.9847 - acc: 0.604 - ETA: 1:26 - loss: 0.9604 - acc: 0.617 - ETA: 1:25 - loss: 0.9369 - acc: 0.629 - ETA: 1:25 - loss: 0.9466 - acc: 0.627 - ETA: 1:24 - loss: 0.9373 - acc: 0.633 - ETA: 1:24 - loss: 0.9179 - acc: 0.638 - ETA: 1:23 - loss: 0.9120 - acc: 0.640 - ETA: 1:22 - loss: 0.9169 - acc: 0.637 - ETA: 1:22 - loss: 0.9050 - acc: 0.642

167/167 [==============================] - ETA: 1:34 - loss: 0.7044 - acc: 0.833 - ETA: 1:34 - loss: 0.8687 - acc: 0.666 - ETA: 1:34 - loss: 0.9159 - acc: 0.611 - ETA: 1:34 - loss: 0.9730 - acc: 0.583 - ETA: 1:33 - loss: 0.9559 - acc: 0.600 - ETA: 1:33 - loss: 0.9848 - acc: 0.583 - ETA: 1:33 - loss: 0.9296 - acc: 0.607 - ETA: 1:33 - loss: 0.9125 - acc: 0.625 - ETA: 1:32 - loss: 0.8952 - acc: 0.638 - ETA: 1:32 - loss: 0.8881 - acc: 0.641 - ETA: 1:31 - loss: 0.8738 - acc: 0.643 - ETA: 1:31 - loss: 0.8625 - acc: 0.645 - ETA: 1:30 - loss: 0.8485 - acc: 0.653 - ETA: 1:30 - loss: 0.8401 - acc: 0.654 - ETA: 1:29 - loss: 0.8357 - acc: 0.661 - ETA: 1:29 - loss: 0.8337 - acc: 0.661 - ETA: 1:28 - loss: 0.8268 - acc: 0.661 - ETA: 1:28 - loss: 0.8203 - acc: 0.671 - ETA: 1:27 - loss: 0.8186 - acc: 0.671 - ETA: 1:26 - loss: 0.8024 - acc: 0.683 - ETA: 1:26 - loss: 0.7873 - acc: 0.690 - ETA: 1:25 - loss: 0.7772 - acc: 0.693 - ETA: 1:25 - loss: 0.7823 - acc: 0.692 - ETA: 1:24 - loss: 0.7690 - acc: 0.697

167/167 [==============================] - ETA: 1:38 - loss: 1.6269 - acc: 0.416 - ETA: 1:37 - loss: 1.2041 - acc: 0.541 - ETA: 1:36 - loss: 1.0056 - acc: 0.638 - ETA: 1:36 - loss: 0.9204 - acc: 0.687 - ETA: 1:35 - loss: 0.9565 - acc: 0.683 - ETA: 1:35 - loss: 0.9539 - acc: 0.680 - ETA: 1:34 - loss: 0.9213 - acc: 0.702 - ETA: 1:34 - loss: 0.8961 - acc: 0.708 - ETA: 1:33 - loss: 0.9578 - acc: 0.675 - ETA: 1:32 - loss: 0.9120 - acc: 0.683 - ETA: 1:32 - loss: 0.9014 - acc: 0.681 - ETA: 1:31 - loss: 0.9108 - acc: 0.673 - ETA: 1:31 - loss: 0.8824 - acc: 0.679 - ETA: 1:30 - loss: 0.9032 - acc: 0.660 - ETA: 1:30 - loss: 0.9115 - acc: 0.650 - ETA: 1:29 - loss: 0.9025 - acc: 0.656 - ETA: 1:28 - loss: 0.9020 - acc: 0.656 - ETA: 1:28 - loss: 0.8966 - acc: 0.657 - ETA: 1:27 - loss: 0.9025 - acc: 0.657 - ETA: 1:27 - loss: 0.8926 - acc: 0.666 - ETA: 1:26 - loss: 0.8899 - acc: 0.666 - ETA: 1:25 - loss: 0.8808 - acc: 0.670 - ETA: 1:25 - loss: 0.8861 - acc: 0.670 - ETA: 1:24 - loss: 0.8845 - acc: 0.673

167/167 [==============================] - ETA: 1:37 - loss: 0.8025 - acc: 0.666 - ETA: 1:37 - loss: 0.6761 - acc: 0.791 - ETA: 1:38 - loss: 0.7105 - acc: 0.750 - ETA: 1:37 - loss: 0.6316 - acc: 0.791 - ETA: 1:37 - loss: 0.7233 - acc: 0.750 - ETA: 1:36 - loss: 0.7359 - acc: 0.736 - ETA: 1:35 - loss: 0.7427 - acc: 0.714 - ETA: 1:34 - loss: 0.7454 - acc: 0.718 - ETA: 1:34 - loss: 0.7624 - acc: 0.713 - ETA: 1:33 - loss: 0.7527 - acc: 0.716 - ETA: 1:32 - loss: 0.7650 - acc: 0.712 - ETA: 1:32 - loss: 0.7697 - acc: 0.701 - ETA: 1:31 - loss: 0.7591 - acc: 0.698 - ETA: 1:31 - loss: 0.7407 - acc: 0.708 - ETA: 1:30 - loss: 0.7508 - acc: 0.705 - ETA: 1:29 - loss: 0.7452 - acc: 0.703 - ETA: 1:29 - loss: 0.7478 - acc: 0.705 - ETA: 1:28 - loss: 0.7493 - acc: 0.703 - ETA: 1:26 - loss: 0.7524 - acc: 0.699 - ETA: 1:26 - loss: 0.7518 - acc: 0.697 - ETA: 1:25 - loss: 0.7588 - acc: 0.692 - ETA: 1:25 - loss: 0.7628 - acc: 0.691 - ETA: 1:25 - loss: 0.7720 - acc: 0.686 - ETA: 1:24 - loss: 0.7727 - acc: 0.685

...

167/167 [==============================] - ETA: 1:45 - loss: 0.9484 - acc: 0.583 - ETA: 1:41 - loss: 0.8470 - acc: 0.625 - ETA: 1:41 - loss: 0.8085 - acc: 0.666 - ETA: 1:39 - loss: 0.7901 - acc: 0.666 - ETA: 1:39 - loss: 0.7597 - acc: 0.666 - ETA: 1:39 - loss: 0.7705 - acc: 0.638 - ETA: 1:38 - loss: 0.8090 - acc: 0.619 - ETA: 1:37 - loss: 0.8049 - acc: 0.614 - ETA: 1:36 - loss: 0.8085 - acc: 0.620 - ETA: 1:35 - loss: 0.7975 - acc: 0.641 - ETA: 1:34 - loss: 0.7897 - acc: 0.643 - ETA: 1:34 - loss: 0.7721 - acc: 0.652 - ETA: 1:33 - loss: 0.7516 - acc: 0.666 - ETA: 1:33 - loss: 0.7534 - acc: 0.666 - ETA: 1:32 - loss: 0.7670 - acc: 0.655 - ETA: 1:32 - loss: 0.7627 - acc: 0.656 - ETA: 1:31 - loss: 0.7749 - acc: 0.652 - ETA: 1:30 - loss: 0.7610 - acc: 0.657 - ETA: 1:30 - loss: 0.7626 - acc: 0.662 - ETA: 1:29 - loss: 0.7555 - acc: 0.666 - ETA: 1:28 - loss: 0.7613 - acc: 0.662 - ETA: 1:28 - loss: 0.7562 - acc: 0.670 - ETA: 1:27 - loss: 0.7479 - acc: 0.677 - ETA: 1:27 - loss: 0.7525 - acc: 0.677

167/167 [==============================] - ETA: 1:40 - loss: 0.7039 - acc: 0.666 - ETA: 1:37 - loss: 0.7524 - acc: 0.666 - ETA: 1:44 - loss: 0.8063 - acc: 0.638 - ETA: 1:46 - loss: 0.8887 - acc: 0.583 - ETA: 1:43 - loss: 0.8538 - acc: 0.616 - ETA: 1:42 - loss: 0.8806 - acc: 0.597 - ETA: 1:41 - loss: 0.8366 - acc: 0.631 - ETA: 1:39 - loss: 0.8216 - acc: 0.645 - ETA: 1:38 - loss: 0.8466 - acc: 0.620 - ETA: 1:36 - loss: 0.8464 - acc: 0.616 - ETA: 1:35 - loss: 0.8360 - acc: 0.628 - ETA: 1:34 - loss: 0.8511 - acc: 0.618 - ETA: 1:33 - loss: 0.8290 - acc: 0.634 - ETA: 1:33 - loss: 0.8086 - acc: 0.648 - ETA: 1:32 - loss: 0.8446 - acc: 0.633 - ETA: 1:32 - loss: 0.8538 - acc: 0.625 - ETA: 1:31 - loss: 0.8347 - acc: 0.632 - ETA: 1:31 - loss: 0.8274 - acc: 0.638 - ETA: 1:30 - loss: 0.8363 - acc: 0.636 - ETA: 1:30 - loss: 0.8267 - acc: 0.645 - ETA: 1:29 - loss: 0.8183 - acc: 0.654 - ETA: 1:28 - loss: 0.8033 - acc: 0.662 - ETA: 1:27 - loss: 0.7852 - acc: 0.673 - ETA: 1:27 - loss: 0.7703 - acc: 0.680

167/167 [==============================] - ETA: 1:38 - loss: 0.5055 - acc: 0.833 - ETA: 1:37 - loss: 0.6442 - acc: 0.750 - ETA: 1:37 - loss: 0.7417 - acc: 0.694 - ETA: 1:37 - loss: 0.7065 - acc: 0.708 - ETA: 1:37 - loss: 0.7202 - acc: 0.716 - ETA: 1:36 - loss: 0.7249 - acc: 0.708 - ETA: 1:35 - loss: 0.6826 - acc: 0.738 - ETA: 1:34 - loss: 0.6854 - acc: 0.739 - ETA: 1:34 - loss: 0.6774 - acc: 0.731 - ETA: 1:33 - loss: 0.6918 - acc: 0.725 - ETA: 1:33 - loss: 0.6891 - acc: 0.719 - ETA: 1:32 - loss: 0.6719 - acc: 0.729 - ETA: 1:32 - loss: 0.7094 - acc: 0.705 - ETA: 1:31 - loss: 0.7220 - acc: 0.696 - ETA: 1:30 - loss: 0.7338 - acc: 0.688 - ETA: 1:30 - loss: 0.7195 - acc: 0.697 - ETA: 1:29 - loss: 0.7283 - acc: 0.696 - ETA: 1:29 - loss: 0.7196 - acc: 0.703 - ETA: 1:28 - loss: 0.7048 - acc: 0.714 - ETA: 1:27 - loss: 0.7027 - acc: 0.716 - ETA: 1:27 - loss: 0.7018 - acc: 0.714 - ETA: 1:26 - loss: 0.6984 - acc: 0.719 - ETA: 1:26 - loss: 0.7110 - acc: 0.710 - ETA: 1:25 - loss: 0.7240 - acc: 0.708

167/167 [==============================] - ETA: 1:35 - loss: 0.9730 - acc: 0.666 - ETA: 1:34 - loss: 0.7645 - acc: 0.750 - ETA: 1:34 - loss: 0.8537 - acc: 0.666 - ETA: 1:33 - loss: 0.8344 - acc: 0.687 - ETA: 1:33 - loss: 0.8329 - acc: 0.683 - ETA: 1:32 - loss: 0.7857 - acc: 0.708 - ETA: 1:31 - loss: 0.7778 - acc: 0.714 - ETA: 1:31 - loss: 0.8196 - acc: 0.697 - ETA: 1:30 - loss: 0.7591 - acc: 0.731 - ETA: 1:30 - loss: 0.7782 - acc: 0.733 - ETA: 1:29 - loss: 0.7408 - acc: 0.750 - ETA: 1:29 - loss: 0.7561 - acc: 0.729 - ETA: 1:28 - loss: 0.7442 - acc: 0.737 - ETA: 1:28 - loss: 0.7500 - acc: 0.732 - ETA: 1:27 - loss: 0.7623 - acc: 0.722 - ETA: 1:26 - loss: 0.7733 - acc: 0.713 - ETA: 1:26 - loss: 0.7650 - acc: 0.715 - ETA: 1:25 - loss: 0.7956 - acc: 0.703 - ETA: 1:25 - loss: 0.7944 - acc: 0.701 - ETA: 1:24 - loss: 0.7720 - acc: 0.716 - ETA: 1:24 - loss: 0.7746 - acc: 0.718 - ETA: 1:23 - loss: 0.7928 - acc: 0.712 - ETA: 1:22 - loss: 0.7900 - acc: 0.713 - ETA: 1:22 - loss: 0.7699 - acc: 0.725

167/167 [==============================] - ETA: 1:38 - loss: 0.6253 - acc: 0.750 - ETA: 1:38 - loss: 0.7963 - acc: 0.666 - ETA: 1:38 - loss: 0.7530 - acc: 0.694 - ETA: 1:37 - loss: 0.6746 - acc: 0.729 - ETA: 1:38 - loss: 0.6451 - acc: 0.750 - ETA: 1:38 - loss: 0.6469 - acc: 0.750 - ETA: 1:37 - loss: 0.6197 - acc: 0.761 - ETA: 1:35 - loss: 0.6288 - acc: 0.750 - ETA: 1:34 - loss: 0.6464 - acc: 0.740 - ETA: 1:33 - loss: 0.6575 - acc: 0.733 - ETA: 1:32 - loss: 0.6503 - acc: 0.742 - ETA: 1:32 - loss: 0.6692 - acc: 0.722 - ETA: 1:32 - loss: 0.6898 - acc: 0.717 - ETA: 1:31 - loss: 0.6905 - acc: 0.720 - ETA: 1:30 - loss: 0.7229 - acc: 0.705 - ETA: 1:29 - loss: 0.7276 - acc: 0.697 - ETA: 1:29 - loss: 0.7335 - acc: 0.696 - ETA: 1:28 - loss: 0.7152 - acc: 0.703 - ETA: 1:28 - loss: 0.7079 - acc: 0.706 - ETA: 1:28 - loss: 0.7281 - acc: 0.691 - ETA: 1:28 - loss: 0.7416 - acc: 0.690 - ETA: 1:27 - loss: 0.7511 - acc: 0.685 - ETA: 1:27 - loss: 0.7469 - acc: 0.684 - ETA: 1:26 - loss: 0.7382 - acc: 0.691

In [63]:
model.load_weights('saved_models/weights.cancer.hdf5')

In [64]:
# get index of predicted clas for each image in test set
class_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(class_predictions)==np.argmax(test_targets, axis=1))/len(class_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 61.8333%


In [65]:
import csv
class_probabilities = [model.predict(np.expand_dims(tensor, axis=0)) for tensor in test_tensors]

In [66]:
import csv
file_index = 0
with open('results.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=',')
    csv_writer.writerow(['Id'] + ['task_1'] + ['task_2'])
    for result in class_probabilities:
        csv_writer.writerow([test_files[file_index]] + [result[0,0]] + [result[0,2]])
        file_index = file_index + 1